<a href="https://colab.research.google.com/github/khemanth2709/INNOSPHERE/blob/main/forgery_detection_model_h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gradio as gr
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
import os

# Error Level Analysis Function
def apply_ela(image_path, quality=75):
    try:
        # Read original image
        if isinstance(image_path, str):
            original = cv2.imread(image_path)
            if original is None:
                raise ValueError(f"Could not read image: {image_path}")
        else:
            original = image_path.copy()

        # Save image with compression
        temp_path = "temp_compressed.jpg"
        cv2.imwrite(temp_path, original, [cv2.IMWRITE_JPEG_QUALITY, quality])

        # Read compressed image
        compressed = cv2.imread(temp_path)

        # Remove temporary file
        if os.path.exists(temp_path):
            os.remove(temp_path)

        if original.shape == compressed.shape:
            # Calculate absolute difference
            ela_image = cv2.absdiff(original, compressed)
            # Enhance the difference
            ela_image = cv2.convertScaleAbs(ela_image, alpha=2.5)
            return ela_image
        else:
            raise ValueError("Image dimensions don't match after compression")

    except Exception as e:
        print(f"Error in ELA: {e}")
        return None

# Prediction Function
def predict_single_image(image, model):
    try:
        # Convert PIL to OpenCV
        image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        original_image = cv2.resize(image_cv, (128, 128))

        # Apply ELA
        ela_image = apply_ela(image_cv)
        if ela_image is None:
            return "Error in ELA processing", None, None

        ela_image = cv2.resize(ela_image, (128, 128))
        ela_normalized = ela_image / 255.0
        ela_normalized = np.expand_dims(ela_normalized, axis=0)

        # Make prediction
        prediction = model.predict(ela_normalized, verbose=0)
        confidence = np.max(prediction)
        predicted_class = np.argmax(prediction)

        class_names = ['✅ AUTHENTIC', '❌ TAMPERED']
        result = {
            'prediction': class_names[predicted_class],
            'confidence': float(confidence),
            'authentic_prob': float(prediction[0][0]),
            'tampered_prob': float(prediction[0][1])
        }

        return result, original_image, ela_image

    except Exception as e:
        return f"Error: {str(e)}", None, None

# Load the model
try:
    model = load_model('forgery_detection_model.h5')
    MODEL_LOADED = True
    print("✅ Model loaded successfully!")
except Exception as e:
    MODEL_LOADED = False
    print(f"❌ Model loading failed: {e}")

# Gradio Interface Function
def predict_interface(image):
    if not MODEL_LOADED:
        return "❌ Model not loaded. Please check if model file exists.", None, "Error"

    if image is None:
        return "Please upload an image", None, "Waiting..."

    result, original, ela = predict_single_image(image, model)

    if isinstance(result, str):
        return result, None, "Error"

    # Create visualization
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))

    # Original image
    ax1.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    ax1.set_title('Uploaded Image', fontweight='bold')
    ax1.axis('off')

    # ELA image
    ax2.imshow(cv2.cvtColor(ela, cv2.COLOR_BGR2RGB))
    ax2.set_title('ELA Analysis', fontweight='bold')
    ax2.axis('off')

    # Prediction chart
    classes = ['Authentic', 'Tampered']
    probabilities = [result['authentic_prob'], result['tampered_prob']]
    colors = ['#2ecc71', '#e74c3c']

    bars = ax3.barh(classes, probabilities, color=colors, height=0.6)
    ax3.set_xlim(0, 1)
    ax3.set_xlabel('Probability', fontweight='bold')
    ax3.set_title(f"Prediction: {result['prediction']}\nConfidence: {result['confidence']:.2%}",
                 fontweight='bold', fontsize=12)

    # Add probability text
    for bar, prob in zip(bars, probabilities):
        width = bar.get_width()
        ax3.text(width + 0.01, bar.get_y() + bar.get_height()/2,
                f'{prob:.2%}', ha='left', va='center', fontweight='bold', fontsize=11)

    plt.tight_layout()

    # Result text
    result_text = f"""
    ## 🔍 Analysis Results

    **Prediction:** {result['prediction']}
    **Confidence Level:** {result['confidence']:.2%}
    **Authentic Probability:** {result['authentic_prob']:.2%}
    **Tampered Probability:** {result['tampered_prob']:.2%}

    **Interpretation:** {'The image appears to be REAL/ORIGINAL' if 'AUTHENTIC' in result['prediction'] else 'The image shows signs of TAMPERING/MANIPULATION'}
    """

    confidence_gauge = f"🎯 Confidence: {result['confidence']:.2%}"

    return result_text, fig, confidence_gauge

# Create Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="AI Image Forgery Detector") as demo:
    gr.Markdown(
    """
    # 🕵️ AI Image Forgery Detection System
    #### *Detect manipulated images using Error Level Analysis (ELA) and Deep Learning*

    **How it works:** This system analyzes JPEG compression artifacts using Error Level Analysis (ELA)
    and uses a Convolutional Neural Network to detect inconsistencies that indicate image tampering.
    """)

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(label="📤 Upload Image", type="pil", height=300)
            analyze_btn = gr.Button("🔍 Analyze Image", variant="primary", size="lg")

        with gr.Column():
            result_output = gr.Markdown(label="📊 Analysis Results")
            confidence_output = gr.Textbox(label="🎯 Confidence Score")
            plot_output = gr.Plot(label="🖼️ Visual Analysis")

    analyze_btn.click(
        fn=predict_interface,
        inputs=image_input,
        outputs=[result_output, plot_output, confidence_output]
    )

    gr.Markdown("""
    ### 📖 How to Use:
    1. Upload any JPG/JPEG image
    2. Click "Analyze Image"
    3. View detailed analysis results
    4. Check confidence score and ELA visualization

    **Note:** Works best with JPG/JPEG images. Higher resolution images give better results.
    """)

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

❌ Model loading failed: [Errno 2] Unable to synchronously open file (unable to open file: name = 'forgery_detection_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8afae4c15c9d3b282b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
